<a href="https://colab.research.google.com/github/DonghaeSuh/kw-AI-hackathon/blob/main/kfold_data_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [25]:
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K
from numpy.random import seed

In [3]:
train=pd.read_csv('data/train_features.csv')
train_label=pd.read_csv('data/train_labels.csv')
test=pd.read_csv('data/test_features.csv')
sub=pd.read_csv('data/sample_submission.csv')

In [4]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

train["acc_total"]=(train["acc_x"]**2+train["acc_y"]**2+train["acc_z"]**2)**0.5
test["acc_total"]=(test["acc_x"]**2+test["acc_y"]**2+test["acc_z"]**2)**0.5

In [5]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [6]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:29<00:00, 104.58it/s]


In [7]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 114.38it/s]


In [8]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [9]:
train=pd.concat(train_dt)

In [10]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:10<00:00, 293.76it/s]


In [11]:
test=pd.concat(test_dt)

In [12]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:02<00:00, 389.73it/s]


In [13]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [14]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [15]:
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [16]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [17]:
series_train.shape, series_test.shape

((3125, 600, 20), (782, 600, 20))

In [18]:
labels = train_label['label'].to_numpy()

In [19]:
import tensorflow as tf

In [20]:
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [21]:
from tensorflow import keras

In [26]:
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

# 모델을 만들어 주는 함수
# 기존 base에서 overfitting이 심해, dropout을 늘림(아직 제출은 안해봄)
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(128, 9, padding='same', input_shape=[600, 20]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(256, 6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(128, 3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [27]:
# checkpoint path
# 중간중간 모델의 weight를 저장할 경로 설정
ckpt_name = 'k_fold_cnn_sh_ckpt.hdf5'
checkpoint_dir_path = 'checkpoint'
checkpoint_path = os.path.join('checkpoint', ckpt_name)

# check checkpoint paht
# 경로가 없으면 생성함
if not(os.path.exists(checkpoint_dir_path)):
  os.mkdir(checkpoint_dir_path)

# callback 함수 목록
callbacks_list = [
    # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
    tf.keras.callbacks.ModelCheckpoint(
        filepath = checkpoint_path,
        monitor='val_loss',
        mode='min',
        save_weights_only=True,
        save_best_only=True
    ),
    # 8번 동안 val_loss의 향상이 없으면 훈련 종료
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        mode='min',
        verbose=1, 
        patience=8
    ),
    tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5)
]

In [28]:
from sklearn.model_selection import StratifiedKFold

In [29]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, random_state=42, shuffle=True)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    accs.append(acc)
    results.append(result)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################


2021-09-30 11:35:24.521487: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-09-30 11:35:24.552045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 11:35:24.552548: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-09-30 11:35:24.552570: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-09-30 11:35:24.553791: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-09-30 11:35:24.554798: I tensorflow/stream_executor/platform/default/

Epoch 1/100


2021-09-30 11:35:25.947545: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-09-30 11:35:26.074449: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


44/44 [==============================] - 2s 34ms/step - loss: 2.9113 - accuracy: 0.4630 - val_loss: 4.5585 - val_accuracy: 0.1021
Epoch 2/100
44/44 [==============================] - 1s 29ms/step - loss: 2.0850 - accuracy: 0.5359 - val_loss: 2.2229 - val_accuracy: 0.5206
Epoch 3/100
44/44 [==============================] - 1s 28ms/step - loss: 1.9061 - accuracy: 0.5477 - val_loss: 1.9147 - val_accuracy: 0.5420
Epoch 4/100
44/44 [==============================] - 1s 27ms/step - loss: 1.7714 - accuracy: 0.5637 - val_loss: 1.7920 - val_accuracy: 0.5715
Epoch 5/100
44/44 [==============================] - 1s 28ms/step - loss: 1.6588 - accuracy: 0.5818 - val_loss: 1.6896 - val_accuracy: 0.5871
Epoch 6/100
44/44 [==============================] - 1s 30ms/step - loss: 1.5499 - accuracy: 0.6070 - val_loss: 1.6113 - val_accuracy: 0.5971
Epoch 7/100
44/44 [==============================] - 1s 31ms/step - loss: 1.4548 - accuracy: 0.6255 - val_loss: 1.4705 - val_accuracy: 0.6298
Epoch 8/100
44/44 

...

44/44 [==============================] - 1s 27ms/step - loss: 0.0863 - accuracy: 0.9925 - val_loss: 0.2538 - val_accuracy: 0.9374
Epoch 77/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0821 - accuracy: 0.9922 - val_loss: 0.2432 - val_accuracy: 0.9392
Epoch 78/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0768 - accuracy: 0.9940 - val_loss: 0.2637 - val_accuracy: 0.9317
Epoch 79/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0762 - accuracy: 0.9936 - val_loss: 0.2544 - val_accuracy: 0.9389
Epoch 80/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0714 - accuracy: 0.9964 - val_loss: 0.2424 - val_accuracy: 0.9399
Epoch 81/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0689 - accuracy: 0.9943 - val_loss: 0.2406 - val_accuracy: 0.9435
Epoch 82/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0710 - accuracy: 0.9954 - val_loss: 0.2539 - val_accuracy: 0.9389
Epoch 83/100

Epoch 30/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4903 - accuracy: 0.8713 - val_loss: 0.6345 - val_accuracy: 0.8365
Epoch 31/100
44/44 [==============================] - 1s 27ms/step - loss: 0.4684 - accuracy: 0.8795 - val_loss: 0.5658 - val_accuracy: 0.8475
Epoch 32/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4567 - accuracy: 0.8749 - val_loss: 0.5402 - val_accuracy: 0.8532
Epoch 33/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4445 - accuracy: 0.8813 - val_loss: 0.5483 - val_accuracy: 0.8560
Epoch 34/100
44/44 [==============================] - 1s 26ms/step - loss: 0.4385 - accuracy: 0.8813 - val_loss: 0.5859 - val_accuracy: 0.8294
Epoch 35/100
44/44 [==============================] - 1s 27ms/step - loss: 0.4193 - accuracy: 0.8873 - val_loss: 0.5428 - val_accuracy: 0.8535
Epoch 36/100
43/44 [============================>.] - ETA: 0s - loss: 0.3974 - accuracy: 0.8964
Epoch 00036: ReduceLROnPlateau reducing learni

44/44 [==============================] - 1s 26ms/step - loss: 0.1108 - accuracy: 0.9915 - val_loss: 0.2555 - val_accuracy: 0.9317
Epoch 86/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1091 - accuracy: 0.9900 - val_loss: 0.2406 - val_accuracy: 0.9339
Epoch 87/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1055 - accuracy: 0.9922 - val_loss: 0.2411 - val_accuracy: 0.9346
Epoch 88/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1027 - accuracy: 0.9908 - val_loss: 0.2460 - val_accuracy: 0.9325
Epoch 89/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1008 - accuracy: 0.9929 - val_loss: 0.2437 - val_accuracy: 0.9335
Epoch 90/100
44/44 [==============================] - 1s 26ms/step - loss: 0.0989 - accuracy: 0.9929 - val_loss: 0.2380 - val_accuracy: 0.9360
Epoch 91/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0950 - accuracy: 0.9932 - val_loss: 0.2306 - val_accuracy: 0.9374
Epoch 92/100

Epoch 39/100
43/44 [============================>.] - ETA: 0s - loss: 0.3223 - accuracy: 0.9237
Epoch 00039: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
44/44 [==============================] - 1s 26ms/step - loss: 0.3252 - accuracy: 0.9229 - val_loss: 0.5767 - val_accuracy: 0.8550
Epoch 00039: early stopping
44/44 [==============================] - 0s 6ms/step - loss: 0.2421 - accuracy: 0.9385
Fold 9
####################
Epoch 1/100
44/44 [==============================] - 1s 29ms/step - loss: 2.9013 - accuracy: 0.4547 - val_loss: 4.2607 - val_accuracy: 0.1184
Epoch 2/100
44/44 [==============================] - 1s 26ms/step - loss: 2.0805 - accuracy: 0.5332 - val_loss: 2.1637 - val_accuracy: 0.5208
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 1.8898 - accuracy: 0.5499 - val_loss: 1.9321 - val_accuracy: 0.5496
Epoch 4/100
44/44 [==============================] - 1s 27ms/step - loss: 1.7498 - accuracy: 0.5656 - val_loss: 1.8072 - val_

43/44 [============================>.] - ETA: 0s - loss: 0.1870 - accuracy: 0.9644
Epoch 00054: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
44/44 [==============================] - 1s 26ms/step - loss: 0.1882 - accuracy: 0.9648 - val_loss: 0.5157 - val_accuracy: 0.8706
Epoch 55/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1646 - accuracy: 0.9698 - val_loss: 0.2973 - val_accuracy: 0.9221
Epoch 56/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1484 - accuracy: 0.9769 - val_loss: 0.2995 - val_accuracy: 0.9229
Epoch 57/100
44/44 [==============================] - 1s 26ms/step - loss: 0.1412 - accuracy: 0.9776 - val_loss: 0.2948 - val_accuracy: 0.9257
Epoch 58/100
44/44 [==============================] - 1s 28ms/step - loss: 0.1305 - accuracy: 0.9808 - val_loss: 0.2694 - val_accuracy: 0.9317
Epoch 59/100
44/44 [==============================] - 1s 27ms/step - loss: 0.1258 - accuracy: 0.9840 - val_loss: 0.2783 - val_accuracy:

In [30]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [31]:
# 제출물 생성
sub.iloc[:, 1:] = pred
sub.to_csv('sub/submission_3_sh.csv', index=False)

In [32]:
accs

[0.9509246349334717,
 0.9267425537109375,
 0.9278094172477722,
 0.9288762211799622,
 0.9263868927955627,
 0.9285460114479065,
 0.9591183662414551,
 0.9377888441085815,
 0.9384998083114624,
 0.9388552904129028]

In [33]:
results

[0.2028590887784958,
 0.2905593514442444,
 0.28234779834747314,
 0.2850564122200012,
 0.2898443639278412,
 0.28586068749427795,
 0.16538652777671814,
 0.24256178736686707,
 0.24208040535449982,
 0.24323362112045288]